In [1]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing import image
import numpy as np

In [2]:
train_dir = 'chest_xray/train'
test_dir = 'chest_xray/test'
val_dir = 'chest_xray/val'

# Image dimensions and batch size
image_size = (224, 224)
batch_size = 32

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [4]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 5216 images belonging to 2 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 624 images belonging to 2 classes.


In [6]:
val_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 16 images belonging to 2 classes.


In [7]:
class ImageDataset(Dataset):
    def __init__(self, image_generator):
        self.image_generator = image_generator
        self.data = image_generator[0][0]  # Extract images
        self.labels = image_generator[0][1]  # Extract labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Convert image data to PyTorch tensor
        image = torch.tensor(self.data[idx], dtype=torch.float32)
        label = torch.tensor(int(self.labels[idx]), dtype=torch.long)
        return image, label

# Assuming you have an ImageDataGenerator (train_generator)
train_dataset = ImageDataset(train_generator)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [13]:
class ANNModel(nn.Module):
    def __init__(self, input_size):
        super(ANNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)  # Input layer
        self.fc2 = nn.Linear(256, 256)
        #self.fc3 = nn.Linear(256, 256)
        #self.fc4 = nn.Linear(256, 256)
        #self.fc5 = nn.Linear(256, 256)
        self.fc6 = nn.Linear(256, 256)
        self.fc7 = nn.Linear(256, 1)           # Output layer for binary classification
        self.sigmoid = nn.Sigmoid()           # Sigmoid activation for binary classification
    
    def forward(self, x):
        x = torch.flatten(x, 1)  # Flatten image to 1D (for ANN)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        #x = torch.relu(self.fc3(x))
        #x = torch.relu(self.fc4(x))
        #x = torch.relu(self.fc5(x))
        x = torch.relu(self.fc6(x))
        x = self.sigmoid(self.fc7(x))
        return x

In [21]:
# Model initialization
model = ANNModel(input_size=224*224*3)  # Assuming images are 224x224
criterion = nn.BCELoss()  # Binary cross-entropy loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training Loop
for epoch in range(100):  # Number of epochs
    for images, labels in train_loader:
        # Move data to GPU if available
        if torch.cuda.is_available():
            images, labels = images.cuda(), labels.cuda()
            model = model.cuda()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.squeeze(), labels.float())

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/10], Loss: {loss.item():.4f}')


Epoch [1/10], Loss: 0.6897
Epoch [2/10], Loss: 0.4755
Epoch [3/10], Loss: 0.4590
Epoch [4/10], Loss: 0.4413
Epoch [5/10], Loss: 0.4429
Epoch [6/10], Loss: 0.3972
Epoch [7/10], Loss: 0.3976
Epoch [8/10], Loss: 0.3619
Epoch [9/10], Loss: 0.3516
Epoch [10/10], Loss: 0.3093
Epoch [11/10], Loss: 0.3101
Epoch [12/10], Loss: 0.2677
Epoch [13/10], Loss: 0.2637
Epoch [14/10], Loss: 0.2300
Epoch [15/10], Loss: 0.2186
Epoch [16/10], Loss: 0.1944
Epoch [17/10], Loss: 0.1787
Epoch [18/10], Loss: 0.1649
Epoch [19/10], Loss: 0.1439
Epoch [20/10], Loss: 0.1366
Epoch [21/10], Loss: 0.1170
Epoch [22/10], Loss: 0.1096
Epoch [23/10], Loss: 0.0962
Epoch [24/10], Loss: 0.0854
Epoch [25/10], Loss: 0.0787
Epoch [26/10], Loss: 0.0671
Epoch [27/10], Loss: 0.0618
Epoch [28/10], Loss: 0.0550
Epoch [29/10], Loss: 0.0476
Epoch [30/10], Loss: 0.0439
Epoch [31/10], Loss: 0.0386
Epoch [32/10], Loss: 0.0336
Epoch [33/10], Loss: 0.0309
Epoch [34/10], Loss: 0.0274
Epoch [35/10], Loss: 0.0239
Epoch [36/10], Loss: 0.0218
E

In [ ]:
def train(model, optimizer, loss_fn, train_loader, val_loader,
epochs=20, device="cpu"):
for epoch in range(epochs):
training_loss = 0.0
valid_loss = 0.0
model.train()
for batch in train_loader:
optimizer.zero_grad()
inputs, target = batch
inputs = inputs.to(device)
target = targets.to(device)
output = model(inputs)
loss = loss_fn(output, target)
loss.backward()
optimizer.step()
training_loss += loss.data.item()
training_loss /= len(train_iterator)
model.eval()
num_correct = 0
num_examples = 0
for batch in val_loader:
inputs, targets = batch
inputs = inputs.to(device)
output = model(inputs)
targets = targets.to(device)
loss = loss_fn(output,targets)
valid_loss += loss.data.item()
correct = torch.eq(torch.max(F.softmax(output), dim=1)[1],
target).view(-1)
num_correct += torch.sum(correct).item()
num_examples += correct.shape[0]
valid_loss /= len(valid_iterator)
print('Epoch: {}, Training Loss: {:.2f},
Validation Loss: {:.2f},
accuracy = {:.2f}'.format(epoch, training_loss,
valid_loss, num_correct / num_examples))

In [25]:
def evaluate(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation
        for images, labels in test_loader:
            # Move data to GPU if available
            if torch.cuda.is_available():
                images, labels = images.cuda(), labels.cuda()

            # Forward pass to get predictions
            outputs = model(images)
            predicted = (outputs > 0.5).float()  # Threshold at 0.5 for binary classification

            # Calculate total and correct predictions
            total += labels.size(0)
            correct += (predicted.squeeze() == labels).sum().item()

    accuracy = correct / total * 100  # Accuracy as a percentage
    print(f'Accuracy: {accuracy:.2f}%')



In [26]:
test_dataset = ImageDataset(test_generator)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [27]:
evaluate(model, test_loader)

Accuracy: 59.38%
